## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units
import importlib

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)
import dataset_comp
import snapshot

In [ ]:
importlib.reload(dataset_comp)
importlib.reload(snapshot)

# Spatial Distibution of Low-mass Subhalos

## Motivation

Here, I make a projection plot of the centres of potential of subhalos. I'm particularly looking at the low-mass halos and inspecting the beads-on-a-string structures that appear in the curv-p082 model. Many of these beads likely do not represent physical halos, since they are very low in mass, but are rather traces of the non-physical fragmentation of physical filamentary structures, due to numerical noise. 

Thus, we can use these figures to find a kind of upper mass limit for the subhalos that we trust to be non-numerical.

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_MR_curvaton_p082_fix']
names = ['plain-LCDM', 'spec-p082']

m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 2000 # Maximum distance from LG centre for isolated

Set the low-mass threshold. Subhalos with masses above or below this limit are plotted on different subplots.

In [ ]:
lowm = 7

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns}

### Compute Projections

First, get centres of potential of subhalos and compute them wrt. the LG centre. Then, find two orthogonal planes that intersect both M31 and MW. Denote the two vectors normal to these planes by n1 and n2. The two (arbitrary) parameters a and b determine the orientation angle of the plane normal to n1. 

In [ ]:
for name, sim_data in data.items():
    snap = sim_data['snapshot']
    cops = snap.get_subhalos("CentreOfPotential")
    m31 = sim_data['M31_identifier']
    mw = sim_data['MW_identifier']
    LG_centre = dataset_comp.compute_LG_centre(snap, m31, mw)

    # Centre on LG:
    cops = dataset_comp.periodic_wrap(snap, LG_centre, cops) - LG_centre

    # Convert units:
    cops = cops * units.cm.to(units.Mpc)

    # Compute the unit separation vector:
    cop_m31 = cops[snap.index_of_halo(m31[0], m31[1])]
    cop_mw = cops[snap.index_of_halo(mw[0], mw[1])]
    r = cop_m31 - cop_mw
    r = r / np.linalg.norm(r)

    # Compute a unit vector orthogonal to r, n1:
    a = 1; b = 1
    n1 = np.array([-1/r[0] * (a*r[1] + b*r[2]), a, b])
    n1 = n1 / np.linalg.norm(n1)

    # Compute another unit vector orthogonal to both r and n1:
    n2 = np.cross(r, n1)

    # Project onto the plane orthogonal to n1:
    data[name]['Proj1'] = {
        'COP' : np.column_stack((np.dot(cops, r), np.dot(cops, n2))),
        'M31' : {
            'COP' : np.array((np.dot(cop_m31, r), np.dot(cop_m31, n2)))
        },
        'MW' : {
            'COP' : np.array((np.dot(cop_mw, r), np.dot(cop_mw, n2)))
        }
    }

    # Project onto the plane orthogonal to n2:
    data[name]['Proj2'] = {
        'COP' : np.column_stack((np.dot(cops, r), np.dot(cops, n1))),
        'M31' : {
            'COP' : np.array((np.dot(cop_m31, r), np.dot(cop_m31, n1)))
        },
        'MW' : {
            'COP' : np.array((np.dot(cop_mw, r), np.dot(cop_mw, n1)))
        }
    }

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data['snapshot']
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    data[name]['Vmax'] = vmax

    # Split into satellites:    
    m31 = sim_data['M31_identifier']
    mw = sim_data['MW_identifier']
    if distinction == 'by_r':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance(
            snap, m31, mw)
    elif distinction == 'by_gn':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, m31, mw)
        
    # Compute masking arrays:
    mask_m31 = masks_sat[0]
    mask_mw = masks_sat[1]
    mask_subhalo = np.logical_and(np.arange(mask_m31.size) != snap.index_of_halo(m31[0], m31[1]),
                                  np.arange(mask_m31.size) != snap.index_of_halo(mw[0], mw[1]))
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    
    # Mask low mass subhalos:
    mask_lowm = vmax < lowm
    
    # Prune potential spurious:
    mask_nonzero_vmax = dataset_comp.prune_vmax(snap)
    mask_m31 = mask_m31[mask_nonzero_vmax]
    mask_mw = mask_mw[mask_nonzero_vmax]
    mask_lum = mask_lum[mask_nonzero_vmax]
    mask_dark = mask_dark[mask_nonzero_vmax]
    mask_subhalo = mask_subhalo[mask_nonzero_vmax]
    
    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Isolated': mask_isol,
        'Subhalos': mask_subhalo,
        'Luminous': mask_lum,
        'Dark': mask_dark,
        'LowMass': mask_lowm
    }

---

## Plot 

Plot subhalos with vmax < lowm in the upper row, and the more massive ones in the lower one. Plot two columns for both simulations.

First, set figure parameters:

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

# Set marker size limits:
smin = 0.001; smax = 3
mmin = 0; mmax = 10**10
# Iterate through simulations to find :
for sim_data in data.values():    
    mmax = np.max(sim_data['Vmax'][sim_data['Selections']['Subhalos']])
    mmin = np.min(sim_data['Vmax'][sim_data['Selections']['Subhalos']])

# Select projection plane:
x = 0; y = 1

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

# Set marker size limits:
slow = 0.001; smax = 2
mlow = lowm; mmax = 0

# Iterate through simulations to find largest value of vmax:
for sim_data in data.values():    
    mmax = max(
        mmax, np.max(sim_data['Vmax'][sim_data['Selections']['Subhalos']])
    )

# Select projection plane:
x = 0; y = 1

In [ ]:
print(mlow, mmax)

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(ncols=len(data), nrows=2, sharey='all', sharex='all', 
                         subplot_kw=dict(box_aspect=1), figsize=(4.9,5))
plt.subplots_adjust(wspace=0.04)
plt.subplots_adjust(hspace=0.04)

axes[0,0].set_ylabel('$y [\mathrm{Mpc}]$')
axes[1,0].set_xlabel('$x [\mathrm{Mpc}]$')
axes[1,0].set_ylabel('$y [\mathrm{Mpc}]$')
axes[1,1].set_xlabel('$x [\mathrm{Mpc}]$')
    
# Set axis limits:
ax_size = 5
axes[1, 0].set_xlim(-ax_size/2, ax_size/2)
axes[1, 0].set_ylim(-ax_size/2, ax_size/2)

# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):
    
    axes[0, i].set_title(key)    
    
    # Set text size:
#     s = 14
#     axes[0, i].title.set_size(s)
#     axes[1, i].xaxis.label.set_size(s)
#     axes[0, 0].yaxis.label.set_size(s)
#     axes[1, 0].yaxis.label.set_size(s)
    
    def mass_to_marker(x):
        return 1/(mmax-mmin) * (smax*(x-mmin) - smin*(x-mmax))
    
    # Plot spurious low-mass subhalos in the upper row:
    mask_dark = np.logical_and(sim_data['Selections']['Dark'],
                          sim_data['Selections']['LowMass'])
    axes[0,i].scatter(sim_data['Proj1']['COP'][mask_dark][:,0], 
                      sim_data['Proj1']['COP'][mask_dark][:,1],
                      c='black',
                      alpha=0.7,
                      s=mass_to_marker(sim_data['Vmax'][mask_dark]))
    
    mask_lum = np.logical_and(sim_data['Selections']['Luminous'],
                              sim_data['Selections']['LowMass'])    
    axes[0,i].scatter(sim_data['Proj1']['COP'][mask_lum][:,0], 
                      sim_data['Proj1']['COP'][mask_lum][:,1],
                      facecolors='none', 
                      edgecolors='blue',
                      alpha=0.7,
                      s=10*mass_to_marker(sim_data['Vmax'][mask_lum]))

    
    # Plot physically representative more massive halos in the lower row:
    mask_dark = np.logical_and.reduce([sim_data['Selections']['Subhalos'],
                                       sim_data['Selections']['Dark'],
                                       np.logical_not(sim_data['Selections']['LowMass'])])
    axes[1,i].scatter(sim_data['Proj1']['COP'][mask_dark][:,0], 
                      sim_data['Proj1']['COP'][mask_dark][:,1],
                      c='gray',
                      alpha=0.7,
                      s=10*mass_to_marker(sim_data['Vmax'][mask_dark]))

    mask_lum = np.logical_and.reduce([sim_data['Selections']['Subhalos'],
                                      sim_data['Selections']['Luminous'],
                                      np.logical_not(sim_data['Selections']['LowMass'])])    
    axes[1,i].scatter(sim_data['Proj1']['COP'][mask_lum][:,0], 
                      sim_data['Proj1']['COP'][mask_lum][:,1],
                      facecolors='none', 
                      edgecolors='blue',
                      alpha=0.5,
                      s=10*mass_to_marker(sim_data['Vmax'][mask_lum]))
    
    

In [ ]:
# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):
    
    # Plot central halo centres in all subplots:    
    axes[0,i].scatter(sim_data['Proj1']['M31']['COP'][0], 
                      sim_data['Proj1']['M31']['COP'][1],
                      c='red', s=4)
    axes[0,i].scatter(sim_data['Proj1']['MW']['COP'][0], 
                      sim_data['Proj1']['MW']['COP'][1],
                      c='red', s=4)
    axes[1,i].scatter(sim_data['Proj1']['M31']['COP'][0],
                      sim_data['Proj1']['M31']['COP'][1],
                      c='red', s=4)
    axes[1,i].scatter(sim_data['Proj1']['MW']['COP'][0], 
                      sim_data['Proj1']['MW']['COP'][1],
                      c='red', s=4)
    
    # Plot 2Mpc circles (for isolated):
    def circle(centre_x, centre_y, r, n):
        t = 4*np.pi/n * np.arange(n/2)
        x = centre_x + r * np.cos(t)
        y = centre_y + r * np.sin(t)
        return x, y

    lw = 0.3
    x_circ, y_circ = circle(0, 0, 2, 10000)
    axes[0,i].plot(x_circ, y_circ, c='black', linestyle='dashed', linewidth=lw)
    axes[1,i].plot(x_circ, y_circ, c='black', linestyle='dashed', linewidth=lw)
    
# Add text box:
axes[0,1].text(0.9, 0.1, "$v_\mathrm{{max}} < {} \mathrm{{km/s}}$".format(lowm),
               transform=axes[0,1].transAxes, ha="right", va="bottom")
axes[1,1].text(0.9, 0.1, "$v_\mathrm{{max}} > {} \mathrm{{km/s}}$".format(lowm),
               transform=axes[1,1].transAxes, ha="right", va="bottom")

fig

### Save the Figure

In [ ]:
# Construct saving location:
filename = 'Satellite_locations_low_mass'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
home = os.path.abspath(os.path.join('..'))
path = os.path.join(home,'Figures', 'MediumResolution')
filename = os.path.join(path, filename)
    
fig.savefig(filename, dpi=300)